<a href="https://colab.research.google.com/github/hurtado1820/InteligenciaArtificial/blob/main/sistema_experto_basico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 import sqlite3  #Importo librería para manejo de bases de datos

con = sqlite3.connect('mydatabase.db')  #Definir conexión a una base de datos
cursor = con.cursor()  #Cursor para interactuar con la base de datos

In [2]:
#Creación de las tablas
cursor.execute("CREATE TABLE hechos(id_h INTEGER PRIMARY KEY, nombre TEXT)")
cursor.execute('''CREATE TABLE reglas(id_r INTEGER PRIMARY KEY, id_anteced INTEGER, id_consec INTEGER,
                FOREIGN KEY(id_anteced) REFERENCES hechos(id_h),FOREIGN KEY(id_consec) REFERENCES hechos(id_h) )''')

In [3]:
#Adicionar información en la tabla de hechos 
cursor.execute("INSERT INTO hechos VALUES(1,'fiebre')")
cursor.execute("INSERT INTO hechos VALUES(2,'tos')")
cursor.execute("INSERT INTO hechos VALUES(3,'malestar')")
cursor.execute("INSERT INTO hechos VALUES(4,'dolor_de_garganta')")
cursor.execute("INSERT INTO hechos VALUES(5,'gripa')")
cursor.execute("INSERT INTO hechos VALUES(6,'faringitis')")

In [4]:
#Adicionar información en la tabla de reglas
cursor.execute("INSERT INTO reglas VALUES(1,1,5)")
cursor.execute("INSERT INTO reglas VALUES(2,2,5)")
cursor.execute("INSERT INTO reglas VALUES(3,3,5)")
cursor.execute("INSERT INTO reglas VALUES(4,3,6)")
cursor.execute("INSERT INTO reglas VALUES(5,4,6)")

In [5]:
#Vista contenido tabla hechos
cursor.execute("SELECT * FROM hechos")
result = cursor.fetchall()
print(result)

[(1, 'fiebre'), (2, 'tos'), (3, 'malestar'), (4, 'dolor_de_garganta'), (5, 'gripa'), (6, 'faringitis')]


In [6]:
#Vista contenido tabla reglas
cursor.execute("SELECT * FROM reglas")
result = cursor.fetchall()
print(result)

[(1, 1, 5), (2, 2, 5), (3, 3, 5), (4, 3, 6), (5, 4, 6)]


In [7]:
#Generación de agenda
agenda = {}

In [84]:
#Busco el consecuente (enfermedad) y lleno la agenda
def manejoAgenda(sintoma):
  cursor.execute("SELECT id_consec FROM reglas WHERE id_anteced = '%i'" %sintoma)
  enfermedad_id = cursor.fetchall()
  if len(enfermedad_id) == 1:
    cursor.execute("SELECT nombre FROM hechos WHERE id_h = '%i'" %enfermedad_id[0][0])
    nomb_enfermedad = cursor.fetchall()
    if nomb_enfermedad[0][0] in agenda:
      agenda[nomb_enfermedad[0][0]] += 1
    else:
      agenda[nomb_enfermedad[0][0]] = 1  

  if len(enfermedad_id) == 2:
    cursor.execute("SELECT nombre FROM hechos WHERE id_h = '%i'" %enfermedad_id[0][0])
    nomb_enfermedad = cursor.fetchall()
    if nomb_enfermedad[0][0] in agenda:
      agenda[nomb_enfermedad[0][0]] += 1
    else:
      agenda[nomb_enfermedad[0][0]] = 1

    cursor.execute("SELECT nombre FROM hechos WHERE id_h = '%i'" %enfermedad_id[1][0])
    nomb_enfermedad = cursor.fetchall()
    if nomb_enfermedad[0][0] in agenda:
      agenda[nomb_enfermedad[0][0]] += 1
    else:
      agenda[nomb_enfermedad[0][0]] = 1  
  
  print(agenda)


#Tomo el identificador del sintoma proporcionado
def sistema(entrada):
  cursor.execute("SELECT id_h FROM hechos WHERE nombre = '%s'" %entrada)
  sintoma = cursor.fetchall()
  if len(sintoma) == 1:
    valor_sintoma = sintoma[0][0]
    manejoAgenda(valor_sintoma)   


#Ciclo de pregunta
while True:
  entrada = input("Ingrese un sintoma: ")
  if entrada != "nada":
    sistema(entrada)
  else:
    break  

Ingrese un sintoma: fiebre
{'gripa': 1}
Ingrese un sintoma: malestar
{'gripa': 2, 'faringitis': 1}
Ingrese un sintoma: tos
{'gripa': 3, 'faringitis': 1}
Ingrese un sintoma: nada


In [85]:
#Revisión de Agenda
num = list(agenda.values())
nombres = list(agenda.keys())

#Caso de agenda sin sintomas
if len(num) == 0:
  print("Agenda vacía")

elif len(num) > 0:   
  #Sintoma con mayor valor y ver si hay empate   
  maximo = max(num)
  cantidad = num.count(maximo)
  if cantidad > 1:
   print("No está definida la enfermedad, hay empate")
  elif cantidad == 1:
    enfermedad_pos = num.index(maximo)
    enfermedad_nombre = nombres[enfermedad_pos]
    print("El paciente tiene: ",enfermedad_nombre)  

agenda.clear()

El paciente tiene:  gripa
